In [11]:
!wget raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

URL transformed to HTTPS due to an HSTS policy
--2024-02-21 05:09:23--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-02-21 05:09:24 (20.7 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [12]:
with open('input.txt', 'r') as file:
    text_data = file.read()

In [13]:
print(text_data[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [14]:
print(len(text_data))



1115394


In [15]:
# get the unique text characters
unique_chars = sorted(list(set(text_data)))

vocab_size = len(unique_chars)
# join the unique characters to form a string
unique_chars = ''.join(unique_chars)
print(unique_chars)
print(vocab_size)



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [16]:
# encoder and decoder

def encode_text(text):
    return [unique_chars.index(c) for c in text]

def decode_text(codes):
    return ''.join(unique_chars[c] for c in codes)

print(encode_text('hello, world!'))
print(decode_text(encode_text('hello, world!')))


[46, 43, 50, 50, 53, 6, 1, 61, 53, 56, 50, 42, 2]
hello, world!


In [17]:
import torch
data = torch.tensor(encode_text(text_data), dtype=torch.long)
print(data.shape, data.dtype)
print(str(data[:100]))


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [18]:
# split into train and validate
split_index = int(len(data) * 0.9)
train_data = data[:split_index]
validate_data = data[split_index:]

the train_data that we run on needs to be 1 bigger than the block_size because within transformer training, each element is used to predict the next element. So, from the 8th element, we need to predict the 9th. Thus, for 8 training datapoints, we need 9 datapoints.

In [19]:
block_size=8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [20]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x)
print(y)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} then target is {target}")



tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])
when input is tensor([18]) then target is 47
when input is tensor([18, 47]) then target is 56
when input is tensor([18, 47, 56]) then target is 57
when input is tensor([18, 47, 56, 57]) then target is 58
when input is tensor([18, 47, 56, 57, 58]) then target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) then target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) then target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then target is 58


In [24]:
torch.manual_seed(1337)
batch_size=4# how many independent sequences are processed in parallel
block_size=8# what is the maximum context length for predictions?

def get_batch(split):
    data = train_data if split == 'train' else validate_data
    # generate (batch_size) random starting indices for the sequences.
    starts = torch.randint(len(data)-block_size, (batch_size,))
    # get the data for each of the (batch_size) sequences
    batch = torch.stack([data[i:i+block_size] for i in starts])
    batch_offset = torch.stack([data[i+1:i+1+block_size] for i in starts])
    return batch, batch_offset

batch, batch_offset = get_batch('train')
print("inputs:")
print(batch)
print(batch.shape)
print("targets:")
print(batch_offset)
print(batch_offset.shape)

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = batch[b, 0:t+1]
        target = batch_offset[b, t]
        print(f"when input is {context} then target is {target}")


inputs:
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
targets:
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
torch.Size([4, 8])
when input is tensor([24]) then target is 43
when input is tensor([24, 43]) then target is 58
when input is tensor([24, 43, 58]) then target is 5
when input is tensor([24, 43, 58,  5]) then target is 57
when input is tensor([24, 43, 58,  5, 57]) then target is 1
when input is tensor([24, 43, 58,  5, 57,  1]) then target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) then target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) then target is 39
when input is tensor([44]) then target is 53
when input is tensor([44, 53]) then target is 56
when input is tensor([44, 53, 56]) then tar

In [ ]:
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])